In [ ]:
%matplotlib inline
%load_ext cudf.pandas
import pandas as pd
import re
import sys; sys.path.append('./')
import log_search
import os
import matplotlib.pyplot as plt


# Define a function to extract the pattern 'B_1 A 2.4 0.31 ms' or similar
def extract_pattern(log_value):
    if isinstance(log_value, str):
        match = re.search(r'[BR]_\d+ A (\d+\.\d+%?|\d+%?|\d+) \d+(\.\d+)? ms', log_value)
        if match:
            return match.group(0)
    return None
def get_filenames_by_group(df):
    # Group by 'SliceID', 'CellID', and 'extracted_log'
    grouped = df.groupby(['SliceID', 'CellID', 'extracted_log'])['filename'].apply(list)
    
    return grouped.reset_index(name='filenames')

def read_csvs_in_directory(filenames, log_data):
    # Dictionary to store dataframes
    dfs = []
    for filename in filenames:
        abffilename = filename[0:8] + '.abf'
        # Filter the dataframe for the target filename
        target_row = log_data[log_data['filename'] == abffilename]
        
        # Extract the log entry for the target row
        log_entry = target_row.iloc[0]['log']
        slice_id = target_row.iloc[0]['SliceID']
        cell_id = target_row.iloc[0]['CellID']
        
        csv_file = os.path.join(csv_directory, filename)
        try:
            df = pd.read_csv(csv_file)
            df['filename'] = filename[0:8]
            df['SliceID'] = slice_id
            df['CellID'] = cell_id
            dfs.append(df)
        except Exception as e:
            print(e)
    dfs = pd.concat(dfs, axis=0, sort=True)
    return dfs

def max_abs_im_scaled_during_period(data, start_time, end_time):
    """
    Calculate the maximum absolute value of Im_scaled for each filename during a specified period.
    
    Parameters:
    data (pd.DataFrame): The original dataframe containing the data.
    start_time (float): The start time of the period.
    end_time (float): The end time of the period.
    
    Returns:
    pd.DataFrame: A dataframe with filenames and their corresponding max absolute Im_scaled values during the specified period.
    """
    # Filter the data based on the specified time period
    filtered_data = data[(data['Time (seconds)'] >= start_time) & (data['Time (seconds)'] <= end_time)]
    
    # Calculate the maximum absolute value of Im_scaled for each filename within the specified period
    max_abs_im_scaled_period = filtered_data.groupby('filename')['Im_scaled'].apply(lambda x: x.abs().max()).reset_index()
    
    # Rename the columns for better readability
    max_abs_im_scaled_period.columns = ['filename', 'max_abs_im_scaled']
    
    return max_abs_im_scaled_period

def calculate_mean_peak_times(file_paths, file_color):
    """
    Calculate the mean of Peak Time for a given ADC_Name based on the color passed for a list of file paths.
    
    Parameters:
    file_paths (list of str): The list of paths to the CSV files.
    file_color (str): The color to determine the ADC_Name ('Blue' or 'Red').
    
    Returns:
    pd.DataFrame: A DataFrame with file names (without extension) and their corresponding mean Peak Times.
    """
    mean_peak_times = []
    file_names = []
    
    # Determine the ADC_Name based on the color
    adc_name = 'Stim_2' if file_color == 'Blue' else 'Stim_3'
    
    for file_path in file_paths:
        # Extract the file name without extension
        file_name = os.path.splitext(os.path.basename(file_path))[0].replace('_peak', '')
        file_names.append(file_name)
        
        # Check if the file exists
        if os.path.exists(file_path):
            # Load the CSV file
            data = pd.read_csv(file_path)
            
            # Filter the data based on the ADC_Name
            filtered_data = data[data['ADC Name'] == adc_name]
            
            # Calculate the mean of Peak Time
            mean_peak_time = filtered_data['Peak Time'].mean()
            
            # Append the mean peak time to the list
            mean_peak_times.append(mean_peak_time)
        else:
            # If the file does not exist, append None to the list
            mean_peak_times.append(None)
    
    # Create a DataFrame with file names and mean peak times
    result_df = pd.DataFrame({
        'filename': file_names,
        'mean_peak_time': mean_peak_times
    })
    
    return result_df

In [ ]:
log_filename = '2024.XX.XX.csv'
# Find the file recursively
directory_log_filename = log_search.find_file_recursively("./Data", log_filename)
log_data = pd.read_csv(directory_log_filename)
csv_directory = os.path.dirname(directory_log_filename)

# Apply the function to the 'log' column to extract the pattern
log_data['extracted_log'] = log_data['log'].apply(extract_pattern)

In [ ]:
blue_filenames = log_data['filename'].str.replace('.abf', '_bluestim.csv', regex=False).to_list()
peak_filenames = log_data['filename'].str.replace('.abf', '_peak.csv', regex=False).to_list()
blue_data = read_csvs_in_directory(blue_filenames, log_data)
peak_data = read_csvs_in_directory(peak_filenames, log_data) #結局使ってない?

mean_blue_data = blue_data.groupby(['filename', 'SliceID', 'CellID', 'Time (seconds)']).mean()
mean_blue_data = mean_blue_data.reset_index()

log_data_for_merge = log_data
log_data_for_merge['filename'] = log_data_for_merge['filename'].str.replace('.abf', '')
# Filter rows where 'Time (seconds)' is between 1 ~ 2
filtered_df = mean_blue_data[(mean_blue_data['Time (seconds)'] >= 1) & 
                        (mean_blue_data['Time (seconds)'] <= 1.25)]

# Find the maximum absolute value of 'Im_scaled' for each filename within this period
peak_values_df = filtered_df.groupby('filename').apply(lambda x: x.loc[x['Im_scaled'].abs().idxmax()])
peak_values_df = peak_values_df.reset_index(drop=True)
peak_values_df_log = pd.merge(peak_values_df, log_data_for_merge, on = ['filename', 'SliceID', 'CellID'])
peak_values_df_log.to_csv(os.path.join(csv_directory, 'Peak_data_blue.csv'))

In [ ]:
red_filenames = log_data['filename'].str.replace('.abf', '_redstim.csv', regex=False).to_list()
peak_filenames = log_data['filename'].str.replace('.abf', '_peak.csv', regex=False).to_list()
red_data = read_csvs_in_directory(red_filenames, log_data)
peak_data = read_csvs_in_directory(peak_filenames, log_data) #結局使ってない?

mean_red_data = red_data.groupby(['filename', 'SliceID', 'CellID', 'Time (seconds)']).mean()
mean_red_data = mean_red_data.reset_index()

log_data_for_merge = log_data
log_data_for_merge['filename'] = log_data_for_merge['filename'].str.replace('.abf', '')
# Filter rows where 'Time (seconds)' is between 1 ~ 2
filtered_df = mean_red_data[(mean_red_data['Time (seconds)'] >= 1) & 
                        (mean_red_data['Time (seconds)'] <= 1.25)]

# Find the maximum absolute value of 'Im_scaled' for each filename within this period
peak_values_df = filtered_df.groupby('filename').apply(lambda x: x.loc[x['Im_scaled'].abs().idxmax()])
peak_values_df = peak_values_df.reset_index(drop=True)
peak_values_df_log = pd.merge(peak_values_df, log_data_for_merge, on = ['filename', 'SliceID', 'CellID'])
peak_values_df_log.to_csv(os.path.join(csv_directory, 'Peak_data_red.csv'))

In [ ]:
# Create a new column for the absolute value of 'Im_scaled'
peak_values_df_log['Im_scaled_abs'] = peak_values_df_log['Im_scaled'].abs()

# Group by 'SliceID', 'CellID', and 'extracted_log' and separate the data
grouped_data = peak_values_df_log.groupby(['SliceID', 'CellID', 'extracted_log'])

# Creating a list of dataframes for each group
separated_data = {name: group for name, group in grouped_data}
separated_data = {name: group for name, group in separated_data.items() if len(group) > 1}

In [ ]:
# Correct the naming issue by properly handling the tuple
for group_name, group_data in separated_data.items():
    # Determine colors based on 'log' containing 'PSEM'
    colors = ['red' if 'PSEM' in log else 'blue' for log in group_data['log']]
    
    group_name_str = "_".join(group_name)  # Convert tuple to string for naming the file
    plt.figure(figsize=(10, 6))
    plt.bar(group_data['filename'], group_data['Im_scaled_abs'], color = colors)
    plt.xlabel('Filename')
    plt.ylabel('Im_scaled (Absolute)')
    plt.title(f'Im_scaled (Absolute) vs Filename for Group: {group_name}')
    plt.xticks(rotation=90)
    plt.tight_layout()